In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
import random
# import implicit
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [2]:
articles_df = pd.read_csv('shared_articles.csv')
interactions_df = pd.read_csv('users_interactions.csv')
articles_df.drop(['authorUserAgent', 'authorRegion', 'authorCountry'], axis=1, inplace=True)
interactions_df.drop(['userAgent', 'userRegion', 'userCountry'], axis=1, inplace=True)
articles_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [3]:
articles_df.shape,articles_df.contentId.nunique()

((3122, 10), 3057)

In [4]:
interactions_df.head()

,timestamp,eventType,contentId,personId,sessionId
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714


In [5]:
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.drop('eventType', axis=1, inplace=True)
df = pd.merge(interactions_df[['contentId','personId', 'eventType']], articles_df[['contentId', 'title']], how = 'inner', on = 'contentId')
df.head()

,contentId,personId,eventType,title
0,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem
1,-3499919498720038879,-8845298781299428018,VIEW,Hiri wants to fix the workplace email problem
2,-3499919498720038879,-108842214936804958,VIEW,Hiri wants to fix the workplace email problem
3,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem
4,-3499919498720038879,-1443636648652872475,VIEW,Hiri wants to fix the workplace email problem


In [6]:
df.eventType.value_counts()

VIEW               61043
LIKE                5745
BOOKMARK            2463
COMMENT CREATED     1611
FOLLOW              1407
Name: eventType, dtype: int64

In [7]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 3.0, 
   'FOLLOW': 4.0,
   'COMMENT CREATED': 5.0,  
}

df['eventStrength'] = df['eventType'].apply(lambda x: event_type_strength[x])
df.sample(10)

,contentId,personId,eventType,title,eventStrength
7257,2857117417189640073,-926222002072636259,VIEW,Running GV sprints inside corporates - learn f...,1.0
59130,5949788789046720741,3609194402293569455,VIEW,How to use Docker to run ASP.NET Core apps on ...,1.0
16449,-1995591062742965408,3302556033962996625,COMMENT CREATED,Firebase Test Lab for Android,5.0
50559,957332268361319692,4313045637915476309,VIEW,Tasting Dagger 2 on Android,1.0
21639,-541666025693385823,-6944500707172804068,VIEW,Calm down: Google's use of Material Design on ...,1.0
22783,7168347449326627154,-9009798162809551896,VIEW,NYT: Google To Announce Alexa Competitor Calle...,1.0
30632,1306706744219888810,-1443636648652872475,VIEW,Debugging data transformations using Cloud Dat...,1.0
4926,1738052593226421681,-2165869757092084791,VIEW,Como resolver conflitos no ambiente corporativ...,1.0
55718,-7623502978685822577,670878778036881163,VIEW,"Don't Start Big, Start a Little Snowball",1.0
58372,-5920475612630001479,2754566407772265068,VIEW,How it feels to learn Javascript in 2016,1.0


In [8]:
df.shape,df.eventStrength.unique()

((72269, 5), array([1., 2., 3., 4., 5.]))

In [11]:
df.groupby(['personId', 'contentId', 'title']).sum().reset_index()

,personId,contentId,title,eventStrength
0,-9223121837663643404,-8949113594875411859,"No Brasil, '25% dos celulares ainda são 'Burro...",1.0
1,-9223121837663643404,-8377626164558006982,Bad Writing Is Destroying Your Company's Produ...,1.0
2,-9223121837663643404,-8208801367848627943,Ray Kurzweil: The world isn't getting worse - ...,1.0
3,-9223121837663643404,-8187220755213888616,Organizing for digital acceleration: Making a ...,1.0
4,-9223121837663643404,-7423191370472335463,"Espresso Intents: não é magia, é tecnologia! -...",8.0
...,...,...,...,...
40677,9210530975708218054,8477804012624580461,Gartner: em 2020 não haverá mais aplicativos,10.0
40678,9210530975708218054,8526042588044002101,Cloud Native Part 1: Definition,1.0
40679,9210530975708218054,8856169137131817223,Para inovar é preciso mais do que boas ideias ...,1.0
40680,9210530975708218054,8869347744613364434,Java Garbage Collection Essencial,1.0


In [14]:
df[(df.personId==-9223121837663643404) & (df.contentId==-7423191370472335463)]

,contentId,personId,eventType,title,eventStrength
63204,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63225,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63239,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63240,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63241,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63244,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63246,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0
63257,-7423191370472335463,-9223121837663643404,VIEW,"Espresso Intents: não é magia, é tecnologia! -...",1.0


In [15]:
df = df.drop_duplicates()
grouped_df = df.groupby(['personId', 'contentId', 'title']).sum().reset_index()
grouped_df.sample(10)

,personId,contentId,title,eventStrength
1441,-8823950498314351783,231281690254211527,Saiba como a Cisco está trabalhando com IoT e ...,1.0
27512,3106760029136205156,-8954346068661072425,The technology behind preview photos,1.0
12383,-2979881261169775358,-3348652277274905234,Google usa aprendizado de máquina para melhora...,1.0
27247,2966898457830124144,4313871021753084020,Startups matam. E suas vítimas são as empresas...,1.0
34092,5621833459783231486,-5658245291907121574,Machine Learning and the VP Debate,1.0
14947,-1863544689523892600,-4412848927260591416,New Africa Trend Briefing from TrendWatching |...,1.0
11825,-3275942195366018617,2760335717049716507,"Razorfish, US digital revenues, drag down Publ...",1.0
34388,5663484293775216130,5445999759138709405,Bradesco lança serviço inédito para estimular ...,1.0
37243,7527226129639571966,-2948321821574578861,Quando usar paginação e quando usar scroll inf...,1.0
30053,3649923015832992113,-2097075598039554565,"The Languages, Frameworks and Tools You Should...",3.0


In [16]:
df.shape,grouped_df.eventStrength.unique()

((50910, 5),
 array([ 1.,  3.,  6., 10.,  4.,  8.,  5., 12.,  7., 15.,  2.,  9., 11.,
        13., 14.]))

In [51]:
grouped_df['title'] = grouped_df['title'].astype("category")
grouped_df['personId'] = grouped_df['personId'].astype("category")
grouped_df['contentId'] = grouped_df['contentId'].astype("category")
grouped_df['person_id'] = grouped_df['personId'].cat.codes
grouped_df['content_id'] = grouped_df['contentId'].cat.codes
grouped_df.sample(10)

,personId,contentId,title,eventStrength,person_id,content_id
31684,4209517478660372522,5849490334294127641,gotwarlost/istanbul,1.0,1383,2426
274,-9016528795238256703,-7126520323752764957,"How Google is Remaking Itself as a ""Machine Le...",1.0,17,351
29323,3609194402293569455,2834072258350675251,Plano Nacional de IoT chega em março com parce...,1.0,1319,1925
1378,-8845298781299428018,7024819429467163565,Chrome extension brings Medium's best feature ...,3.0,33,2619
38095,7911347399014555575,1469580151036142903,Don't document your code. Code your documentat...,1.0,1752,1710
9010,-4447090897576014728,-614408900691920886,X-Men | São Paulo vai receber exposição sobre ...,1.0,492,1376
37377,7580644064067188685,6540624159201421051,Bradesco vai lançar novo banco digital: o Next.,1.0,1719,2533
19368,-445337111692715325,-4336355219396419841,Game Of Thrones e suas lições de liderança | E...,1.0,887,793
31858,4254153380739593270,-7518373517401484139,My favorite people and resources to learn Andr...,1.0,1390,297
9626,-4154712816784766009,-133139342397538859,"Novo workaholic trabalha, pratica esportes e t...",1.0,521,1449


In [64]:
len(sparse_content_person.data),len(grouped_df.eventStrength)

(40682, 40682)

In [66]:
grouped_df.person_id.nunique(),grouped_df.personId.nunique()

(1895, 1895)

In [61]:
sparse_content_person = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['content_id'], grouped_df['person_id'])))
sparse_person_content = sparse.csr_matrix((grouped_df['eventStrength'].astype(float), (grouped_df['person_id'], grouped_df['content_id'])))
 
print(sparse_content_person.shape)
print(sparse_person_content.shape)

(2979, 1895)
(1895, 2979)


In [13]:
grouped_df['eventStrength'].head(),grouped_df['content_id'].head(),grouped_df['person_id'].head()

(0    1.0
 1    1.0
 2    1.0
 3    1.0
 4    1.0
 Name: eventStrength, dtype: float64, 0     65
 1    159
 2    187
 3    195
 4    313
 Name: content_id, dtype: int16, 0    0
 1    0
 2    0
 3    0
 4    0
 Name: person_id, dtype: int16)

In [14]:
# np.save('sparse_content_person',sparse_content_person)
# np.save('sparse_person_content',sparse_person_content)
# data = np.load('sparse_content_person.npy',allow_pickle=True)

In [15]:
import pickle

In [16]:
with open('sparse_content_person.pickle','wb') as f:
    pickle.dump(sparse_content_person,f)
with open('sparse_person_content.pickle','wb') as f:
    pickle.dump(sparse_person_content,f)

In [17]:
d = pickle.load(open('d.pickle','rb'))

In [24]:
pct_test=0.2
training_set = sparse_content_person.copy()
nonzero_inds = training_set.nonzero()
nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
num_samples = int(np.ceil(pct_test*len(nonzero_pairs)))
samples = random.sample(nonzero_pairs, num_samples)
content_inds = [index[0] for index in samples]
person_inds = [index[1] for index in samples]
training_set[content_inds, person_inds] = 0
training_set.eliminate_zeros()

In [25]:
training_set.shape

(2979, 1895)

In [39]:
np.unique(sparse_content_person.data)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15.])

In [29]:
len(training_set.data),len(sparse_content_person.data)

(32545, 40682)

In [36]:
import numpy as np
from scipy.sparse import csr_matrix
M = csr_matrix(np.ones([2, 2],dtype=np.int32))
print(M)
print(M.data.shape)
for i in range(np.shape(M)[0]):
    for j in range(np.shape(M)[1]):
        if i==j:
            M[i,j] = 0
print(M)
print(M.data.shape)

  (0, 0)	1
  (0, 1)	1
  (1, 0)	1
  (1, 1)	1
(4,)
  (0, 0)	0
  (0, 1)	1
  (1, 0)	1
  (1, 1)	0
(4,)


In [37]:
M

<2x2 sparse matrix of type '<class 'numpy.intc'>'
	with 4 stored elements in Compressed Sparse Row format>

In [32]:
print(M.data)

[0 1 1 0]


In [33]:
M.eliminate_zeros()

In [34]:
M.data

array([1, 1], dtype=int32)

In [35]:
M

<2x2 sparse matrix of type '<class 'numpy.intc'>'
	with 2 stored elements in Compressed Sparse Row format>